In [1]:
import os
import csv
import pickle
import random
import math
import dicom
import numpy as np
import scipy
from tqdm import tqdm
from natsort import natsorted
from skimage import transform
from sklearn.externals import joblib

import matplotlib.pyplot as plt
%matplotlib inline

#### load and shuffle data

In [2]:
(pts_train, pts_train_val, 
 data_train, label_sys_train, label_dia_train, 
 data_train_val, label_sys_train_val, label_dia_train_val, 
 data_val, data_val_pt_index) = joblib.load('../data_proc/1-data_processed.pkl')

In [3]:
shuffle_index = list(range(data_train.shape[0]))
random.shuffle(shuffle_index)
data_train = data_train[shuffle_index]
label_sys_train = label_sys_train[shuffle_index]
label_dia_train = label_dia_train[shuffle_index]

shuffle_index = list(range(data_train_val.shape[0]))
random.shuffle(shuffle_index)
data_train_val = data_train_val[shuffle_index]
label_sys_train_val = label_sys_train_val[shuffle_index]
label_dia_train_val = label_dia_train_val[shuffle_index]

#### training

In [4]:
from keras.models import Sequential, Graph
from keras.layers.core import Activation, Dense, Dropout, Flatten, Merge, Reshape, Lambda
from keras.layers.core import TimeDistributedDense, TimeDistributedMerge
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ParametricSoftplus, ELU
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianDropout, GaussianNoise
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using Theano backend.


Using gpu device 3: Tesla K80 (CNMeM is disabled)


Continuous Ranked Probability Score (used as loss function)

In [5]:
from keras import backend as K
from theano import tensor as T

# currently for theano, would need modifications for tensorflow

def CRPS(y_true, y_pred):
    return K.mean(K.square(T.cumsum(y_pred, axis=-1) - y_true), axis=-1)

In [7]:
graph = Graph()
graph.add_input(name='input', input_shape=(30, 250, 250))
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv0', input='input')
graph.add_node(BatchNormalization(), name='conv_bn0', input='conv0')
graph.add_node(ELU(), name='conv_activ0', input='conv_bn0')
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv1', input='conv_activ0')
graph.add_node(BatchNormalization(), name='conv_bn1', input='conv1')
graph.add_node(ELU(), name='conv_activ1', input='conv_bn1')
graph.add_node(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'), name='mp0', input='conv_activ1')
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv2', input='mp0')
graph.add_node(BatchNormalization(), name='conv_bn2', input='conv2')
graph.add_node(ELU(), name='conv_activ2', input='conv_bn2')
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv3', input='conv_activ2')
graph.add_node(BatchNormalization(), name='conv_bn3', input='conv3')
graph.add_node(ELU(), name='conv_activ3', input='conv_bn3')
graph.add_node(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'), name='mp1', input='conv_activ3')
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv4', input='mp1')
graph.add_node(BatchNormalization(), name='conv_bn4', input='conv4')
graph.add_node(ELU(), name='conv_activ4', input='conv_bn4')
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv5', input='conv_activ4')
graph.add_node(BatchNormalization(), name='conv_bn5', input='conv5')
graph.add_node(ELU(), name='conv_activ5', input='conv_bn5')
graph.add_node(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'), name='mp2', input='conv_activ5')
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv6', input='mp2')
graph.add_node(BatchNormalization(), name='conv_bn6', input='conv6')
graph.add_node(ELU(), name='conv_activ6', input='conv_bn6')
graph.add_node(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'), name='conv7', input='conv_activ6')
graph.add_node(BatchNormalization(), name='conv_bn7', input='conv7')
graph.add_node(ELU(), name='conv_activ7', input='conv_bn7')
graph.add_node(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'), name='mp3', input='conv_activ7')
graph.add_node(Flatten(), name='conv_flat', input='mp3')

graph.add_node(Dense(1024, activation='relu'), name='systole_fc', input='conv_flat')
graph.add_node(Dropout(0.5), name='systole_dropout', input='systole_fc')
graph.add_node(Dense(600, activation='softmax'), name='systole_softmax', input='systole_dropout')

graph.add_node(Dense(1024, activation='relu'), name='diastole_fc', input='conv_flat')
graph.add_node(Dropout(0.5), name='diastole_dropout', input='diastole_fc')
graph.add_node(Dense(600, activation='softmax'), name='diastole_softmax', input='diastole_dropout')

graph.add_output(name='systole_out', input='systole_softmax')
graph.add_output(name='diastole_out', input='diastole_softmax')

graph.compile(optimizer='rmsprop', loss={'systole_out': CRPS, 'diastole_out': CRPS})

In [10]:
batch_size = 32
nb_epoch = 100
    
checkpointer = ModelCheckpoint(filepath='../model_weights/1-0-convnet_basic_graph.hdf5', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)

history = graph.fit({'input': data_train, 
                     'systole_out': label_sys_train, 
                     'diastole_out': label_dia_train}, 
                    validation_data={'input': data_train_val, 
                                     'systole_out': label_sys_train_val, 
                                     'diastole_out': label_dia_train_val},
                    batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True, verbose=1, 
                    callbacks=[checkpointer, earlystopping])

Train on 5105 samples, validate on 267 samples
Epoch 1/100
5105/5105 [==============================] - 221s - loss: 0.2063 - val_loss: 0.0850
Epoch 00000: val_loss improved from inf to 0.08496, saving model to ../model_weights/1-0-convnet_basic_graph.hdf5
Epoch 2/100
5105/5105 [==============================] - 221s - loss: 0.0953 - val_loss: 0.0825
Epoch 00001: val_loss improved from 0.08496 to 0.08248, saving model to ../model_weights/1-0-convnet_basic_graph.hdf5
Epoch 3/100
5105/5105 [==============================] - 221s - loss: 0.0926 - val_loss: 0.0788
Epoch 00002: val_loss improved from 0.08248 to 0.07875, saving model to ../model_weights/1-0-convnet_basic_graph.hdf5
Epoch 4/100
5105/5105 [==============================] - 221s - loss: 0.0913 - val_loss: 0.0810
Epoch 00003: val_loss did not improve
Epoch 5/100
5105/5105 [==============================] - 221s - loss: 0.0904 - val_loss: 0.0808
Epoch 00004: val_loss did not improve
Epoch 6/100
5105/5105 [========================

#### predict

In [21]:
graph.load_weights('../model_weights/1-0-convnet_basic_graph.hdf5')

preds = graph.predict({'input': data_val}, verbose=1)

preds_sys = np.clip(np.cumsum(preds['systole_out'], axis=-1), 0, 1)
preds_dia = np.clip(np.cumsum(preds['diastole_out'], axis=-1), 0, 1)

In [22]:
preds_sys_pt = {}
preds_dia_pt = {}
for pt in range(501, 701):
    preds_sys_pt[pt] = np.mean(preds_sys[np.where(data_val_pt_index == pt)[0]], axis=0)
    preds_dia_pt[pt] = np.mean(preds_dia[np.where(data_val_pt_index == pt)[0]], axis=0)

In [23]:
with open('../data/sample_submission_validate.csv', 'r') as fi:
    reader = csv.reader(fi)
    header = next(reader)
    
    with open('../submissions/1-0-convnet_basic_graph.csv', 'w') as fo:
        writer = csv.writer(fo, lineterminator='\n')
        writer.writerow(header)
        for rowin in tqdm(reader):
            _id = rowin[0]
            pt, mode = _id.split('_')
            rowout = [_id]
            if mode.lower() == 'systole':
                rowout.extend(preds_sys_pt[int(pt)].tolist())
            elif mode.lower() == 'diastole':
                rowout.extend(preds_dia_pt[int(pt)].tolist())
            else:
                raise
            writer.writerow(rowout)